### Load Envs 

In [1]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
from configs.hd22_config import HD22Configuration
config = HD22Configuration()


### ReAct

In [2]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
config = HD22Configuration()

#### ReAct Agent
from agents.react.react_agent import ReActAgent

GPT_VER = '0613'
react_agent = ReActAgent(reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER), 
                        base_prompt="hwchase17/react-multi-input-json", 
                        schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER), 
                        evaluator=CustomQAEvaluator(llm=QuickAzureChatOpenAI(version='1106')), 
                        stop_words=["Observation"], 
                        thought_word= 'Thought', 
                        action_word='Action',
                        file_logging=True,
                        horizon=3)



react_agent.clear_logs()
react_agent.run_agent_trials(num_trials=2, 
query="Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?",
reference="The former former president of Korea is older. The sum of their age is 99.")



[INFO] Query: Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?
[INFO] Trial 1
[INFO] Thought 1: I need to find out who the former president of Korea is and who the MLB American League MVP of the last season is. I'll start with the former president of Korea.
[INFO] Action 1: YDCSearch(query='former president of Korea')
[INFO] Observation 1: The most recent former president of South Korea is Moon Jae-In, who served from 2017 to 2022.
[INFO] Thought 2: Now I need to find out Moon Jae-In's age.
[INFO] Action 2: YDCSearch(query='Moon Jae-In age')
[INFO] Observation 2: Moon Jae-in was born on January 24, 1953.
[INFO] Thought 3: I need to calculate Moon Jae-In's age based on his birth year. I'll use the current year to do this.
[INFO] Action 3: GetDatetime(IANA_timezone='Asia/Seoul')
[INFO] Observation 3: timezone: Asia/Seoul, current time(year-month-day-hour-minute): 2023-12-1-20-35
[INFO] Ju

### Reflexion ReAct

In [3]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
config = HD22Configuration()

#### Reflexion Agent
from agents.react.react_agent_with_reflexion import ReActWithReflexionAgent
from agents.react.react_fewshot_examples import REACT_REFLECTION_HEADER, REACT_REFLECTIONS
from chains.reflexion_chains import BaseReflexionChain

GPT_VER = '0613'   
reflexion_chains = BaseReflexionChain(
                    reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER),
                    prompt="jet-taekyo-lee/experimental-reflextion", 
                    schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
                    reflexion_examples=REACT_REFLECTIONS
                    )

reflexion_prompt = "jet-taekyo-lee/reflexion-react"
reflexion_agent = ReActWithReflexionAgent(
                        reflexion_chain = reflexion_chains,
                        reflexion_header = REACT_REFLECTION_HEADER,
                        reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER), 
                        base_prompt="hwchase17/react-multi-input-json", 
                        schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER), 
                        evaluator=CustomQAEvaluator(llm=QuickAzureChatOpenAI(version='1106')), 
                        stop_words=["Observation"], 
                        thought_word= 'Thought', 
                        action_word='Action',
                        file_logging=True,
                        horizon=3)



reflexion_agent.clear_logs()
reflexion_agent.run_agent_trials(num_trials=2, 
query="Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?",
reference="The former former president of Korea is older. The sum of their age is 99.")



[INFO] Query: Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?
[INFO] Trial 1
[INFO] Thought 1: I need to find out who the former president of Korea is and who the MLB American League MVP of the last season is. Then, I need to find out their ages. I'll start by finding out who the former president of Korea is.
[INFO] Action 1: YDCSearch(query='former president of Korea')
[INFO] Observation 1: The most recent former president of South Korea is Moon Jae-In, who served from 2017 to 2022.
[INFO] Thought 2: Now that I know the former president of Korea is Moon Jae-In, I need to find out his age.
[INFO] Action 2: YDCSearch(query='Moon Jae-In age')
[INFO] Observation 2: Moon Jae-in was born on January 24, 1953.
[INFO] Thought 3: Now that I know Moon Jae-In was born on January 24, 1953, I can calculate his age. I'll use the current year to do this.
[INFO] Action 3: GetDatetime(IANA_timezone='As

### OpenAI Regacy agent

In [5]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
config = HD22Configuration()

# #### OepnAI Agent
GPT_VER = '0613' 
from agents.openai_legacy.openai_legacy_agent import OpenAILegacyAgent
openai_agent = OpenAILegacyAgent(
    reasoninig_engine = QuickAzureChatOpenAI(version=GPT_VER),
    base_prompt = "jet-taekyo-lee/openai-function-calling-agent",
    schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
    evaluator = CustomQAEvaluator(llm=QuickAzureChatOpenAI(version='1106')),
    action_word = 'Invoking',
    horizon = 3)

openai_agent.clear_logs()
openai_agent.run_agent_trials(num_trials=2, 
query="Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?",
reference="The former former president of Korea is older. The sum of their age is 99.")

[INFO] Query: Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?
[INFO] Trial 1
[INFO] Invoking 1: YDCSearch(query='former president of Korea age')
[INFO] Observation 1: The former president of South Korea, Moon Jae-in, was born on January 24, 1953. As of 2022, he is 69 years old. The current president, Yoon Suk Yeol, was born on December 18, 1960, making him 62 years old as of 2022.
[INFO] Invoking 2: YDCSearch(query='MLB American League MVP last season age')
[INFO] Observation 2: The American League MVP for the 2023 season was Shohei Ohtani from the Los Angeles Angels. He was born on July 5, 1994, which would have made him 29 years old during the 2023 season.
[INFO] Invoking 3: PythonREPL(command='print(69 + 29)')
[INFO] Observation 3: 98
[INFO] Jugdement: You failed to provide an answer because you exceeded the permitted number of reasoning steps. You must give an answer within 3 steps

### Reflexion OpenAI Legacy agent

In [6]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, QuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
config = HD22Configuration()

# # #### Reflexion OepnAI Agent
from agents.openai_legacy.openai_legacy_with_reflexion_agent import OpenAILegacyWithReflexionAgent
from agents.openai_legacy.openai_legacy_fewshot_examples import OPENAI_LEGACY_REFLECTIONS, OPENAI_LEGACY_REFLECTION_HEADER
from chains.reflexion_chains import BaseReflexionChain


GPT_VER = '0613'    
openai_legacy_reflexion_chains = BaseReflexionChain(
                    reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER),
                    prompt="jet-taekyo-lee/openai-reflexion-chain-prompt", 
                    schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
                    reflexion_examples=OPENAI_LEGACY_REFLECTIONS
                    )


openai_legacy_reflexion_agent = OpenAILegacyWithReflexionAgent(
                        reflexion_chain = openai_legacy_reflexion_chains,
                        reflexion_header = OPENAI_LEGACY_REFLECTION_HEADER,
                        reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER), 
                        base_prompt="jet-taekyo-lee/rexlexion-openai-function-calling-agent", 
                        schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
                        evaluator=CustomQAEvaluator(llm=QuickAzureChatOpenAI(version='1106')),
                        action_word='Invoking',
                        file_logging=True,
                        horizon = 3)

openai_legacy_reflexion_agent.clear_logs()
openai_legacy_reflexion_agent.run_agent_trials(num_trials=2, 
query="Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?",
reference="The former former president of Korea is older. The sum of their age is 99.")


[INFO] Query: Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?
[INFO] Trial 1
[INFO] Invoking 1: YDCSearch(query='former president of Korea age')
[INFO] Observation 1: The context provided does not specify which former president of Korea's age you are asking about. There have been multiple former presidents of Korea. Please provide more specific information.
[INFO] Invoking 2: I'm sorry, but there have been multiple former presidents of Korea. Could you please specify which one you're referring to?
[INFO] Answer: I'm sorry, but there have been multiple former presidents of Korea. Could you please specify which one you're referring to?
[INFO] Jugdement: Your answer is incorrect. The correct answer is "The former former president of Korea is older. The sum of their age is 99."
[INFO] Trial 2
[INFO] Reflexion......
[INFO] You have attempted to answer following question before and failed. T

### Parallel Function Calling agent

In [7]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, AsyncQuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
import asyncio
config = HD22Configuration()

#### Parallel Function Calling Agent
GPT_VER = '1106' 
from agents.openai_parallel.openai_parallel_agent import OpenAIParallelAgent

openai_parallel_agent = OpenAIParallelAgent(
    reasoninig_engine = QuickAzureChatOpenAI(version=GPT_VER),
    base_prompt = 'jet-taekyo-lee/parallel-function-calling-agent',
    schemas_and_tools = DefaultSchemasTools(azure_gpt_version=GPT_VER),
    evaluator = CustomQAEvaluator(llm=QuickAzureChatOpenAI('0613')),
    action_word = 'Tool invocations in parallel',
    observation_word = 'Tool-invocations results in parallel',
    use_chat_completion_api = True,
    azure_apenai_client = AsyncQuickAzureOpenAIClient(GPT_VER),
    horizon = 3
)





openai_parallel_agent.clear_logs()
await openai_parallel_agent._run_agent_trials_async(num_trials=2, 
query="Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?",
reference="The former former president of Korea is older. The sum of their age is 99.")



[INFO] Query: Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?
[INFO] Trial 1
[INFO] (step 1-action) Tool invocations in parallel: [Tool 1-> YDCSearch(query=former president of Korea age), Tool 2-> YDCSearch(query=MLB American League MVP last season age)]
[INFO] (step 1-observation) :: [Tool 1-> As of my knowledge cutoff date in April 2023, the most recent former president of South Korea is Moon Jae-in, who was born on January 24, 1953. Therefore, as of April 2023, Moon Jae-in would be 70 years old. Please note that ages will change as time passes, so you may need to calculate the current age based on the birth date provided., Tool 2-> The American League MVP for the 2023 season was Shohei Ohtani of the Los Angeles Angels (LAA). His age at the time of receiving the award is not explicitly stated in the provided context, but since he was born on July 5, 1994, he would have been 29 years 

### Reflexion + Parallel Function Calling agent

In [8]:
import os
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI, AsyncQuickAzureOpenAIClient
from tools.get_tools import DefaultSchemasTools
from langchain import hub
from utils.load_envs import Environ
from chains.qa_evaluators import CustomQAEvaluator
import asyncio
config = HD22Configuration()

#### Reflexion + Parallel Function Calling Agent
from chains.reflexion_chains import BaseReflexionChain
from agents.openai_parallel.openai_parallel_with_reflexion_agent import OpenAIParallelWithReflexionAgent
from agents.openai_parallel.openai_parallel_fewshot_examples import OPENAI_PARALLEL_REFLECTIONS, OPENAI_PARALLEL_REFLECTION_HEADER

GPT_VER = '1106'    
openai_parallel_reflexion_chains = BaseReflexionChain(
                    reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER),
                    prompt="jet-taekyo-lee/openai-parallel-calling-reflexion-chain-prompt", 
                    schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
                    reflexion_examples=OPENAI_PARALLEL_REFLECTIONS
                    )


openai_parallel_reflexion_agent = OpenAIParallelWithReflexionAgent(
                        reflexion_chain = openai_parallel_reflexion_chains,
                        reflexion_header = OPENAI_PARALLEL_REFLECTION_HEADER,
                        reasoninig_engine=QuickAzureChatOpenAI(version=GPT_VER), 
                        base_prompt="jet-taekyo-lee/reflexion-openai-parallel-calling-agent", 
                        schemas_and_tools=DefaultSchemasTools(azure_gpt_version=GPT_VER),
                        evaluator=CustomQAEvaluator(llm=QuickAzureChatOpenAI(version='1106')),
                        action_word = 'Tool invocations in parallel',
                        observation_word = 'Tool-invocations results in parallel',
                        use_chat_completion_api = True,
                        azure_apenai_client = AsyncQuickAzureOpenAIClient(GPT_VER),
                        file_logging=True,
                        horizon = 3)


openai_parallel_reflexion_agent.clear_logs()
await openai_parallel_reflexion_agent._run_agent_trials_async(num_trials=2, 
query="Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?",
reference="The former former president of Korea is older. The sum of their age is 99.")



[INFO] Query: Who is older? The former president of Korea vs. the MLB American League MVP of the last season. What is the result if you sum up the ages of the two?
[INFO] Trial 1
[INFO] (step 1-action) Tool invocations in parallel: [Tool 1-> YDCSearch(query=former president of South Korea), Tool 2-> YDCSearch(query=MLB American League MVP last season)]


RateLimitError for get_function_observation. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 

[INFO] (step 1-observation) :: [Tool 1-> Park Geun-hye is a former president of South Korea who served as the 11th president from 2013 until her impeachment in 2017. She was the first woman to be elected as president of South Korea and the first female president popularly elected as head of state in East Asia. Her presidency ended when she was impeached and later convicted on charges related to corruption. After serving a portion of her prison sentence, Park Geun-hye was released from prison following a pardon in December 2021., Tool 2-> The American League MVP for the 2023 season was Shohei Ohtani of the LA Angels.]
[INFO] (step 2-action) Tool invocations in parallel: [Tool 1-> YDCSearch(query=Park Geun-hye age), Tool 2-> YDCSearch(query=Shohei Ohtani age)]


RateLimitError for get_function_observation. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 

[INFO] (step 2-observation) :: [Tool 1-> Park Geun-hye was born on February 2, 1952. As of my knowledge cutoff in 2023, she would be 71 years old., Tool 2-> Shohei Ohtani is 29 years old.]
[INFO] (step 3-action) Tool invocations in parallel: [Tool 1-> PythonREPL(command=print(71 + 29))]
[INFO] (step 3-observation) :: [Tool 1-> 100]
[INFO] Jugdement: You failed to provide an answer because you exceeded the permitted number of reasoning steps. You must give an answer within 3 steps.
[INFO] Trial 2
[INFO] Reflexion......
[INFO] You have attempted to answer following question before and failed. The following reflection(s) give a plan to avoid failing to answer the question in the same way you did previously. Use them to improve your strategy of correctly answering the given question.
- Reflection: The failure in the previous trial was due to exceeding the permitted number of reasoning steps before providing an answer. The process of identifying the individuals and their ages was correct, b

...

[INFO] (step 1-action) Tool invocations in parallel: [Tool 1-> YDCSearch(query=former president of South Korea), Tool 2-> YDCSearch(query=MLB American League MVP 2022)]
[INFO] (step 1-observation) :: [Tool 1-> Park Geun-hye is a former president of South Korea who served as the 11th president from 2013 until her impeachment in 2017. She was the first woman to be elected as president of South Korea and the first female president popularly elected as head of state in East Asia. Her presidency ended when she was impeached and subsequently convicted on charges related to corruption. After spending nearly five years in prison, Park Geun-hye was released in December 2021 following a presidential pardon., Tool 2-> The American League MVP for the 2022 MLB season was Aaron Judge of the New York Yankees.]


RateLimitError for _invoke_agent_action_async. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 

[INFO] (step 2-action) Tool invocations in parallel: [Tool 1-> YDCSearch(query=Park Geun-hye age), Tool 2-> YDCSearch(query=Aaron Judge age)]


RateLimitError for get_function_observation. -----> Will automatically retry 20 seconds later.
20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 

[INFO] (step 2-observation) :: [Tool 1-> Park Geun-hye was born on February 2, 1952. As of my knowledge cutoff in 2023, she would be 71 years old., Tool 2-> Aaron Judge is 31 years old.]
[INFO] (step 3-action) Tool invocations in parallel: Now I can answer directly without resorting to any tool.
[INFO] Answer: 선생님께서 물으신 대로 하여, 한국의 전 대통령은 만 71세이며, 2022년 MLB 아메리칸 리그 MVP는 만 31세입니다. 두 사람의 나이를 합치면 102세가 되며, 한국의 전 대통령이 MLB 아메리칸 리그 MVP보다 나이가 많습니다.
[INFO] Jugdement: Your answer is incorrect. The correct answer is "The former former president of Korea is older. The sum of their age is 99."
